<a href="https://colab.research.google.com/github/arthurzx22aa/Projeto_Sinais_e_sistemas/blob/main/projetode_sinais_e_sistemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Projeto de sinais e Sistemas**

###**descrição**

O objetivo do nosso projeto construção de uma simples assistente eletrônica, a fim de automatizar a entrega de ração para animais de estimação, através do reconhecimento sonoro do miado ou latido.


In [426]:
#importações
%matplotlib inline
from pathlib import Path
import os
import numpy as np
import random
import sklearn as scikit_learn
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import seaborn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score



from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

##Extrator de features  

In [427]:
def get_features(x, sr):
    features = []

    #Root Mean Square Energy
    rms = librosa.feature.zero_crossing_rate(y=x)
    features.append(rms.mean())

    # Fast Fourier Transform
    fft = np.fft.fft(x)
    features.append(np.mean(np.abs(fft)))

    #Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y=x)
    features.append(zcr.mean())

    #Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=x, sr= sr)
    features.append(mfcc.mean())

    #Spectral bandwidth
    sr = librosa.feature.spectral_rolloff(y=x, sr= sr)
    features.append(sr.mean())

    return features

In [429]:
X = []
Y = []
aux = []

directory = './ESC-50';
sounds = os.listdir(directory)

def extract(sounds, directory):
  X = []
  Y = []
  aux = []

  # pegando uma parte para cada tipo de som
  for sound in sounds:

    #pengando os audios
    path = directory+'/'+sound
    for audio_file in os.listdir(path):
        audio_path = path+'/'+audio_file
        y, sr = librosa.load(audio_path)
        aux.append([y, sr])

    # Escolhendo 50 audios aleatorios de cada tipo de som
    random.shuffle(aux)
    for i in aux:
        X.append(get_features(i[0], i[1]))
        Y.append(sound)

  return X, Y

X, Y = extract(sounds, directory)



In [430]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X = np.array(X)
Y = np.array(Y)

In [431]:
print(X.shape,Y.shape)

(2220, 7) (2220,)


In [432]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 20)

In [437]:
knn_classifier = KNeighborsClassifier(n_neighbors=1)
knn_classifier.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=1)

In [438]:
Y_pred = knn_classifier.predict(X_test)
conf_matrix = confusion_matrix(Y_test, Y_pred)
print('Matriz de Confusão:')
print(conf_matrix)

print("Acurácia:", accuracy_score(Y_test, Y_pred))

print(Y_test)
print("/n")
print(Y_pred)





Matriz de Confusão:
[[354  52]
 [ 36   2]]
Acurácia: 0.8018018018018018
['Other' 'Pet' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Pet' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Pet' 'Pet' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Pet' 'Other' 'Other' 'Pet' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Pet' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' '